# Building Micrograd - Exercises

Exercises from the [building micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0).<br>
This notebook is a slightly re-formatted version of [the original micrograd exercise notebook](https://colab.research.google.com/drive/1FPTx1RXtBfc4MaTkf7viZZD4U2F9gtKN?usp=sharing).

1. Watch the [building micrograd video](https://www.youtube.com/watch?v=VMj-3S1tku0) on YouTube
2. Come back and complete these exercises to level up :)

## Section 1: Derivatives

Here is a mathematical expression that takes $3$ inputs and produces one output:

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from math import sin, cos, exp, log

In [2]:
def f(a, b, c):
  return -a**3 + sin(3*b) - 1.0/c + b**2.5 - a**0.5 # assuming c != 0, a >= 0

print(f(2, 3, 4))

6.336362190988558


Write the function `gradf` that returns the analytical gradient of $f$ i.e. use your skills from calculus to take the derivative, then implement the formula.<br>
If you do not know calculus, feel free to ask wolframalpha, e.g.: https://www.wolframalpha.com/input?i=d%2Fda%28sin%283*a%29%29%29

In [5]:
def gradf(a, b, c):
  grada = -3*a**2 - 0.5*a**(-0.5)
  gradb = 3*cos(3*b) + 2.5*b**1.5
  gradc = 1.0/c**2
  return [grada, gradb, gradc]

# expected answer is the list of
ans = [-12.353553390593273, 10.25699027111255, 0.0625]
yours = gradf(2, 3, 4)
for dim in range(3):
  ok = 'OK' if abs(yours[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {yours[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353553390593273
OK for dim 1: expected 10.25699027111255, yours returns 10.25699027111255
OK for dim 2: expected 0.0625, yours returns 0.0625


Now estimate the gradient numerically without any calculus, using the approximation we used in the video.<br>
You should not call the function `df` from the last cell.

In [8]:
# -----------
h = 1e-6
a, b, c = 2, 3, 4
grada = (f(a+h, b, c) - f(a, b, c))/h
gradb = (f(a, b+h, c) - f(a, b, c))/h
gradc = (f(a, b, c+h) - f(a, b, c))/h
numerical_grad = [grada, gradb, gradc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353559348809995
OK for dim 1: expected 10.25699027111255, yours returns 10.256991666679482
OK for dim 2: expected 0.0625, yours returns 0.062499984743169534


There is an alternative formula that provides a much better numerical approximation to the derivative of a function.<br><br>
Learn about it here: https://en.wikipedia.org/wiki/Symmetric_derivative

**Implement this.**<br><br>
Confirm that for the *same step size $h$* this version gives a better approximation.

In [13]:
# -----------
h = 1e-3
grada = (f(a+h, b, c) - f(a-h, b, c))/(2*h)
gradb = (f(a, b+h, c) - f(a, b-h, c))/(2*h)
gradc = (f(a, b, c+h) - f(a, b, c-h))/(2*h)
numerical_grad2 = [grada, gradb, gradc]
# -----------

for dim in range(3):
  ok = 'OK' if abs(numerical_grad2[dim] - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {numerical_grad2[dim]}")

OK for dim 0: expected -12.353553390593273, yours returns -12.353554401639766
OK for dim 1: expected 10.25699027111255, yours returns 10.256994551617105
OK for dim 2: expected 0.0625, yours returns 0.06250000390650712


## Section 2: Support for Softmax

Without referencing the code/video __too__ much, make the following two cells work.

You'll have to implement (in some cases re-implemented) a number of functions of the Value object, similar to what we've seen in the video.<br>
Instead of the squared error loss, this implements the __negative log likelihood loss__, which is very often used in classification.

In [20]:
# Value class starter code, with many functions taken out
class Value:

  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"

  def __add__(self, other): # exactly as in the video
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')

    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward

    return out
  
  # Multiplication
  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other) # Extension
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += out.grad * other.data # Bugfix
      other.grad += out.grad * self.data # Bugfix
    
    out._backward = _backward
    return out
  
# Negation (special multiplication)
  def __neg__(self): # -self
    return -1 * self
  
# Called if self is on right side of +
  def __radd__(self, other): # other + self
    return self + other
  
  # Called if self is on right side of *
  def __rmul__(self, other): # other * self
    return self * other
  
  # Power (special multiplication)
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers (for now)"
    out = Value(self.data ** other, (self,), f'**{other}')
    
    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    
    out._backward = _backward
    return out
    
  # True division (special multiplication)
  def __truediv__(self, other): # self / other
    return self * other**-1
  
  def exp(self):
    x = self.data
    out = Value(exp(x), (self, ), 'exp')

    def _backward():
      self.grad += out.data * out.grad
    out._backward = _backward

    return out
  
  def log(self):
    x = self.data
    out = Value(log(x), (self,), 'log')

    def _backward():
      self.grad += out.grad / x
      
    out._backward = _backward

    return out

  def backward(self): # exactly as in video
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)

    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [21]:
# this is the softmax function
# https://en.wikipedia.org/wiki/Softmax_function
def softmax(logits):
  counts = [logit.exp() for logit in logits]
  denominator = sum(counts)
  out = [c / denominator for c in counts]
  return out

# this is the negative log likelihood loss function, pervasive in classification
logits = [Value(0.0), Value(3.0), Value(-2.0), Value(1.0)]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()
print(loss.data)

ans = [0.041772570515350445, 0.8390245074625319, 0.005653302662216329, -0.8864503806400986]
for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

2.1755153626167147
OK for dim 0: expected 0.041772570515350445, yours returns 0.041772570515350445
OK for dim 1: expected 0.8390245074625319, yours returns 0.8390245074625319
OK for dim 2: expected 0.005653302662216329, yours returns 0.005653302662216329
OK for dim 3: expected -0.8864503806400986, yours returns -0.8864503806400986


**Verify:** The gradient calculated by using the library `torch` should be exactly the same.

In [37]:
import torch
v1 = torch.Tensor([0.0]); v1.requires_grad = True
v2 = torch.Tensor([3.0]); v2.requires_grad = True
v3 = torch.Tensor([-2.0]); v3.requires_grad = True
v4 = torch.Tensor([1.0]); v4.requires_grad = True

logits = [v1, v2, v3, v4]
probs = softmax(logits)
loss = -probs[3].log() # dim 3 acts as the label for this input example
loss.backward()

for dim in range(4):
  ok = 'OK' if abs(logits[dim].grad.item() - ans[dim]) < 1e-5 else 'WRONG!'
  print(f"{ok} for dim {dim}: expected {ans[dim]}, yours returns {logits[dim].grad}")

OK for dim 0: expected 0.041772570515350445, yours returns tensor([0.0418])
OK for dim 1: expected 0.8390245074625319, yours returns tensor([0.8390])
OK for dim 2: expected 0.005653302662216329, yours returns tensor([0.0057])
OK for dim 3: expected -0.8864503806400986, yours returns tensor([-0.8865])


<b>Congrats! You leveled up!</b>